In [1]:
import tensorflow as tf
import keras


/Users/omar/Documents/GitHub/BeansClassifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2024-03-15 16:48:01.317287: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-15 16:48:01.317309: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-15 16:48:01.317314: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-15 16:48:01.317500: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-15 16:48:01.317516: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
train = keras.utils.image_dataset_from_directory("./data/train/",image_size=(500,500))

Found 1034 files belonging to 3 classes.


In [4]:
test = keras.utils.image_dataset_from_directory("./data/test/",image_size=(500,500))

Found 128 files belonging to 3 classes.


In [5]:
val = keras.utils.image_dataset_from_directory("./data/validation/",image_size=(500,500))

Found 133 files belonging to 3 classes.


In [6]:
for image, label in train:
    print(f'{image.shape}')
    print(f'{label.shape}')
    break

(32, 500, 500, 3)
(32,)


In [7]:
from keras import layers,regularizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [8]:
classes = train.class_names
rlr = ReduceLROnPlateau(patience=3),
es = EarlyStopping(monitor="val_accuracy", patience=5,restore_best_weights=True)

In [9]:
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(500,500,3)),
    layers.RandomFlip("horizontal"),
    layers.Conv2D(32,(3,3),activation="relu",kernel_regularizer=regularizers.l1_l2()),
    layers.MaxPool2D(),
    layers.Conv2D(32,(3,3),activation="relu",kernel_regularizer=regularizers.l1_l2()),
    layers.MaxPool2D(),
    layers.Conv2D(64,(3,3),activation="relu",kernel_regularizer=regularizers.l1_l2()),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(len(classes))
    
])

/Users/omar/Documents/GitHub/BeansClassifier/.venv/lib/python3.9/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 500, 500, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 500, 500, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 498, 498, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 249, 249, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 247, 247, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 123, 123, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 121, 121, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 60, 60, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 230400)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    29,491,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,520,355 (112.61 MB)

 Trainable params: 29,520,355 (112.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
  train,
  validation_data=val,
  epochs=10,
  callbacks=[rlr,es]
)

Epoch 1/10


2024-03-15 16:48:02.298394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 890ms/step - accuracy: 0.3655 - loss: 2.8862 - val_accuracy: 0.6165 - val_loss: 0.8843 - learning_rate: 0.0010
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 28s 844ms/step - accuracy: 0.5626 - loss: 0.8608 - val_accuracy: 0.7669 - val_loss: 0.5703 - learning_rate: 0.0010
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 28s 837ms/step - accuracy: 0.6883 - loss: 0.7301 - val_accuracy: 0.7594 - val_loss: 0.6007 - learning_rate: 0.0010
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 29s 854ms/step - accuracy: 0.7404 - loss: 0.6127 - val_accuracy: 0.7594 - val_loss: 0.6152 - learning_rate: 0.0010
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 32s 970ms/step - accuracy: 0.7678 - loss: 0.5743 - val_accuracy: 0.7744 - val_loss: 0.6829 - learning_rate: 0.0010
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 31s 905ms/step - accuracy: 0.7784 - loss: 0.5383 - val_accuracy: 0.7970 - val_loss: 0.5358 - learning_rate: 1.0000e-04
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 28s 839ms/step - accuracy: 0.8247 - loss: 0.4676 - 

In [13]:
model.evaluate(test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.8333 - loss: 0.4464


[0.4665103554725647, 0.8125]

In [14]:
model.save("beans_classifier.keras")